In [1]:
%autosave 300

Autosaving every 300 seconds


In [2]:
from Cleanser import get_data, convert_time
from functools import reduce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import numpy as np
import seaborn as sns

# sns.set(color_codes=True)
matplotlib.style.use("ggplot")
%matplotlib inline

In [3]:
# print(plt.style.available)
print(matplotlib.matplotlib_fname())

# font_location = "C:Windows/Fonts/NanumBarunGothic.ttf"
font_location = "/Users/RyuQae/Library/Fonts/NanumBarunGothic.otf"
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

print(matplotlib.rcParams["font.family"])

/Users/RyuQae/anaconda/lib/python3.5/site-packages/matplotlib/mpl-data/matplotlibrc
['NanumBarunGothicOTF']


In [4]:
default_period = ['T2', 'T3', 'T4']
default_path = 'nielsen_data_new_20170328/%s'
# dataframe = get_data(period=default_period, path=default_path, time_conversion=True)

# 프로그램시청시간 정수형으로 타입 캐스팅
# dataframe.프로그램시청시간 = dataframe.프로그램시청시간.astype(int)

In [5]:
# tidy_df = dataframe.copy()
# tidy_df.to_pickle('output/tidy_df.pkl')

In [6]:
tidy_df = pd.read_pickle('output/tidy_df.pkl')

# 1. 시청패턴 데이터프레임 제작 : pattern

## 1.1. 60초 이하 시청기록 제거 : overwatch
피플미터는 1분 단위로 정보를 기록하여 60초 미만의 시청시간에 대한 정보는 부정확하므로 제거

In [7]:
overwatch = tidy_df[tidy_df.프로그램시청시간 > 60].reset_index().drop('index', axis=1)

In [8]:
overwatch.shape

(1691901, 18)

## 1.2. 시청 시간대 구분 : when
개인 시청자가 주어진 각 2주간의 시청행위가 어떤 시간대에 일어났는지 나타내는 비율
 - 00시 ~ 06시 : 새벽
 - 06시 ~ 12시 : 오전
 - 12시 ~ 18시 : 오후
 - 18시 ~ 24시 : 밤

In [9]:
overwatch = overwatch.assign(시간대=pd.cut(overwatch.시청시작시간.dt.hour,
        [-1, 5, 11, 17, 23],
        labels=['새벽', '오전', '오후', '밤']))

In [10]:
temp = overwatch[['기간', 'ID','시청시작시간', '시간대','프로그램시청시간']]

In [11]:
new_temp = temp.groupby(['기간', 'ID', '시간대']).프로그램시청시간.apply(sum).reset_index(name='시간대별시청시간')

In [12]:
new_temp['시청시간대비율'] = new_temp.groupby(['기간', 'ID']).시간대별시청시간.apply(lambda x: 100*x/float(x.sum()))

In [13]:
when = pd.pivot_table(new_temp, 
               values='시청시간대비율', 
               index = ['기간', 'ID'], 
               columns = '시간대').replace(np.nan, 0.0)

In [14]:
when['주시청시간대'] = when.idxmax(axis=1)

## 1.3. TV 시청 중 각 장르 시청시간 비율 : gr_pivot

In [15]:
agg1 = {
    '프로그램시청시간' : {
        '시청비율' : 'sum'
    }
}

In [16]:
genre_ratio = overwatch.groupby(['ID', '기간', '대분류'])\
    .agg(agg1).groupby(level=[0,1]).apply(lambda x: 100*x/float(x.sum()))

# Flattening hierarchical columns
genre_ratio.columns = genre_ratio.columns.get_level_values(1)

In [17]:
gr_pivot = pd.pivot_table(genre_ratio.reset_index(), 
                          values = '시청비율', 
                          index = ['기간', 'ID'], 
                          columns = '대분류').replace(np.nan, 0)

## 1.4. TV 시청 중 주요채널 시청시간 비율 : cr_pivot

In [18]:
channel_ratio = overwatch.groupby(['기간', 'ID', '방송사']).agg(agg1).\
        groupby(level=[0,1]).apply(lambda x: 100*x/float(x.sum()))

# Flattening hierarchical columns
channel_ratio.columns = channel_ratio.columns.get_level_values(1)

In [19]:
target_channel = list(tidy_df.방송사.value_counts().head(12).index)

In [20]:
cr_pivot = pd.pivot_table(channel_ratio.reset_index(), 
                          values = '시청비율', 
                          index = ['기간', 'ID'], 
                          columns = '방송사')[target_channel].replace(np.nan, 0)

In [21]:
cr_pivot.sort_values(by='SBS', ascending = False)

방송사                KBS1        MBC    SBS        KBS2        MBN       JTBC  \
기간 ID                                                                         
T3 2403149aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402210ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 1402072ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   1402404ad   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402404ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 2403735ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2402741ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 2402657aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402630ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2402372ab   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2403532aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 2402626ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402558ad   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2402949ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   1402031ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 1402455ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2403842ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 2402748ab   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 2403896ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2403529ab   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 2403265ae   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2402748aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2403209ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 2403394aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402637ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 1402455ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T4 2402971ad   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T3 2403899aa   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
T2 1402688ae   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
   2403149ac   0.000000   0.000000  100.0    0.000000   0.000000   0.000000   
...                 ...        ...    ...         ...        ...        ...   
T3 2403116ad   0.000000   0.000000    0.0  100.000000   0.000000   0.000000   
   2403466ab   1.971390   2.802732    0.0    4.049068   5.122095   0.162477   
   1402745ac   0.000000   0.000000    0.0    0.000000   0.000000   0.000000   
   1402745ad   0.000000   0.000000    0.0    0.000000   0.000000   0.000000   
T4 2403094ad   0.000000   0.000000    0.0    0.000000   0.000000   0.000000   
T2 2403057aa   0.000000   0.000000    0.0    0.000000   0.000000   0.000000   
T4 2403092ac  76.480904   0.000000    0.0   23.519096   0.000000   0.000000   
   2403091ad  11.292835  12.266355    0.0    1.752336   0.000000   0.000000   
T3 1402747aa   0.000000   2.463560    0.0    0.000000   0.532182   1.818289   
   1402747ab   0.000000  74.193548    0.0    0.000000   0.000000   0.000000   
T2 2403055ac   0.000000   1.293011    0.0    0.000000   0.000000   1.293011   
T3 2404139aa   2.825830   7.820674    0.0    3.207698   0.000000   0.000000   
   2404139ab   9.906535  17.349868    0.0    3.582573   0.000000   0.000000   
T2 1402734ac   0.000000   0.000000    0.0    0.000000   0.000000   0.000000   
T3 2403475ac  35.267532  35.865286    0.0   13.121446   0.000000   1.749526   
T2 2403241ad  11.795415   0.000000    0.0   18.134715  32.383420   0.000000   
T3 2403116ac   0.000000   0.000000    0.0  100.000000   0.000000   0.000000   
T4 2403065ab   0.000000   0.000000    0.0    0.000

In [22]:
gr_pivot

대분류                 교육        기타      드라마&영화          보도        스포츠  \
기간 ID                                                                 
T2 1401861aa  0.000000  0.000000   53.271586    4.180853   0.000000   
   1401861ab  0.000000  0.000000   48.021403    4.745918   0.000000   
   1401900ab  0.000000  0.000000   47.161371   22.731464   0.000000   
   1401900ac  0.000000  0.000000   19.592858    0.000000   0.000000   
   1401908aa  0.000000  0.000000    1.435407   55.495557  22.843472   
   1401908ab  0.000000  0.000000    3.702071   39.374174  37.549581   
   1401909aa  0.000000  0.000000    1.153678   63.543405   0.438397   
   1401914aa  0.000000  0.000000   29.881721   43.195404   0.000000   
   1401914ab  0.000000  0.000000  100.000000    0.000000   0.000000   
   1401917aa  0.000000  0.000000   19.785950   19.159108   1.126582   
   1401917ab  2.806699  0.000000   19.805271   35.669220   0.000000   
   1401923aa  0.000000  0.000000   39.413845    0.000000   6.013138   
   1401923ab  0.000000  0.000000    0.000000  100.000000   0.000000   
   1401927aa  0.000000  0.000000    9.223924   46.651346   2.420272   
   1401927ab  0.000000  0.000000    7.455396   42.892573   1.867149   
   1401929aa  0.000000  0.000000   13.387667   24.224625   0.000000   
   1401929ab  0.105003  0.000000   16.489846   23.740725   2.198966   
   1401929ac  0.185533  0.000000   21.666564   16.822689   0.000000   
   1401931aa  0.000000  0.000000    6.526850   39.445927   2.773347   
   1401934aa  0.000000  0.000000   54.298654    7.328684   0.000000   
   1401937aa  0.231583  0.000000   14.511090   16.397311  44.352337   
   1401937ab  0.000000  0.000000   18.601489   43.836201   4.720341   
   1401937ad  0.000000  0.000000   61.482346   38.517654   0.000000   
   1401940aa  0.223225  0.000000   12.547635   41.567146   3.318345   
   1401940ab  0.218134  0.000000   17.635243   44.206719   0.109067   
   1401949aa  0.000000  0.000000   63.728234    3.304904   0.337598   
   1401949ab  0.000000  0.000000   85.217099    0.335721   0.000000   
   1401949ad  0.000000  0.000000   88.034567    0.000000  11.965433   
   1401965aa  0.000000  0.000000   41.173014   10.502391  37.383411   
   1401965ab  0.000000  0.000000    0.000000    0.835538  99.164462   
...                ...       ...         ...         ...        ...   
T4 2404314aa  0.041975  0.000000   23.060393    7.244706   0.000000   
   2404314ab  0.000000  0.000000   12.955929   24.938093   0.000000   
   2404316ab  0.000000  0.000000    3.680200   18.180837   0.000000   
   2404317aa  0.000000  0.000000    9.969382   40.623553  14.503024   
   2404319aa  0.475670  0.000000   12.512887   52.749777   0.000000   
   2404319ab  2.336179  0.000000   41.619432   25.412453   0.000000   
   2404322aa  0.000000  0.000000   39.765468    7.521726   0.000000   
   2404322ab  0.000000  0.000000   43.689332    5.641188   0.000000   
   2404323aa  0.000000  0.000000    1.814323   57.818320  15.172776   
   2404323ab  0.000000  0.000000   35.648357   11.953850   0.000000   
   2404324aa  0.000000  0.000000   22.447799   22.497008   0.000000   
   2404324ab  0.000000  0.000000   30.296048   21.997991   0.000000   
   2404325aa  0.000000  1.071907   34.533584   24.928124   3.616858   
   2404325ab  0.000000  0.612406   43.643591   19.763713   3.259363   
   2404326aa  0.000000  0.711243   39.489103    6.373051   0.625269   
   2404326ab  0.031066  0.108055   48.198190    5.254153   0.270136   
   2404329aa  0.084117  0.122284   57.991014    9.697346  12.158414   
   2404329ab  0.112530  0.163589   44.879526   10.694288  14.010941   
   2404331aa  0.000000  0.000000    8.925526   57.343028  11.963702   
   2404332aa  0.038282  0.000000   28.594984   14.741704   0.000000   
   2404332ab  0.000000  0.000000   62.940280   14.904606   0.000000   
   2404333aa  0.000000  0.000000   12.028059   21.877894   0.000000   
   2404333ab  0.000000  0.000000   11.896869   23.623832   0.000000   
   2404334aa  0.000000  

In [23]:
semi_final = pd.merge(left = cr_pivot.reset_index(), right = gr_pivot.reset_index(), on = ['기간','ID'], how = 'inner')

## 1.5. 그 외 여러가지 시청패턴 : etc
 - 총 시청시간, 시청시간 중간값 및 평균
 - 시청일 수 및 시청요일
 - 채널변경 횟수

### 1.5.1. 총 시청시간, 시청시간 중간값 및 평균, 채널변경 횟수

In [ ]:
agg2 = {
    '프로그램시청시간' : {
        '총시청시간' : 'sum',
        '시청시간중간값' : 'median',
        '평균시청시간' : 'mean'
    },
    '일자' : {
        '시청일자' : lambda x: set(x),
        '시청일수' : lambda y: len(set(y))
    },
    '요일' : {
        '시청요일' : lambda z: set(z)
    },
    '프로그램명' : {
        '채널변경횟수' : 'count'
    }
}

In [ ]:
etc = overwatch.groupby(['기간', 'ID']).agg(agg2)
etc.columns = etc.columns.get_level_values(level=1)

In [ ]:
weekends = {'Sat', 'Sun'}

In [ ]:
def stylecheck(viewdays):
    if len(weekends & viewdays) == 0:
        return '평일'
    elif viewdays < weekends:
        return '주말'
    else:
        return '알수없음'

In [ ]:
etc['라이프스타일'] = etc.시청요일.apply(stylecheck)

In [ ]:
etc.라이프스타일[etc.시청일수 == 14] = '매일'

In [ ]:
final = pd.merge(left = semi_final.reset_index(), right = etc.reset_index(), on=['기간','ID'], how='inner')

In [ ]:
final

In [ ]:
pattern = pd.merge(left = final, right = when.reset_index(), on=['기간', 'ID'], how = 'inner')
# .to_csv('output/final_table.csv', encoding='utf8')

In [ ]:
target_pattern = pattern.loc[pattern.ID.isin(target_audience)].copy()

In [ ]:
target_pattern

## 2. 인구통계학 정보 데이터프레임 제작 : demo_info

In [ ]:
job = """
1	 전문/경영관리
2	 사무직
3	 써비스/기능/생산
4	 자영업
5	 농, 수, 축, 임업
6	 주부
7	 학생
8	 무직
9	 미취학아
:	 기타
""".strip().split('\n')

edu = """
1	 무학
2	 초등학교재학
3	 초등학교졸업
4	 중학교재학
5	 중학교졸업
6	 고등학교재학
7	 고등학교졸업
8	 대학재학
9	 대학졸업
:	 대학원이상
""".strip().split('\n')

earn = """
1	119만원 이하
2	120 ~ 199만원
3	200 ~ 249만원
4	250 ~ 299만원
6	300 ~ 399만원
7	400 ~ 499만원
8	500만원 이상
""".strip().split('\n')

In [ ]:
job_dict = {}
for k, v in map(lambda l: l.split('\t'), job):
    job_dict[k] = v

In [ ]:
edu_dict = {}
for k, v in map(lambda l: l.split('\t'), edu):
    edu_dict[k] = v

In [ ]:
earn_dict = {}
for k, v in map(lambda l: l.split('\t'), earn):
    earn_dict[k] = v

In [ ]:
demo = ['기간', 'ID', '성별', '연령', '직업', '학력', '소득', '가중치']

In [ ]:
agg4 = {
    '성별':lambda x: x.replace({'1':'남', '2':'여'}).iloc[0],
    '연령':lambda x: x.astype(int).max(),
    '직업':lambda x: x.replace(job_dict).iloc[0],
    '학력':lambda x: x.replace(edu_dict).iloc[0],
    '소득':lambda x: x.replace(earn_dict).iloc[0],
    '가중치':lambda x: x.astype(float).mean(),
}

In [ ]:
demo_info = tidy_df[demo].groupby(['ID']).agg(agg4)

In [ ]:
demo_info

# 3. 개인별 시청 변화 추적
 - SBS를 주로 시청한 사람들은 어떤 사람들인가 (기간별로), 뭘 보고 있는가 (채널/장르)
 - SBS를 주로 시청하는 사람들?
 - 보도/드라마/오락을 주로 시청하는 사람들?
 - JTBC를 보게 된 사람들이 어디서 왔는지?
 - 시청 유형이 비슷한 그룹으로 클러스터링


## 3.1. 분석 대상
 - T2, T3, T4 모두 활동한 패널
 - target channel 에서 50% 이상 시청을 나타내는 시청자
 - 2주간 시청일 수 > 2
 - 2주간 총 시청시간 > 3600
 

 - pattern : 시청패턴에 관한 정보
 - demo_info : 시청자의 인구통계학적 정보

## 3.2. 주시청 채널
 - 주로 시청하다 : 12개 메인 방송사 중 가장 많은 비율 시청 (그 비율이 30% 이상)

In [ ]:
target_pattern[target_channel].idxmax(axis=1).value_counts()

각 기간의 active audience 기준: 2주간 등장 3일 이상, 총 1시간 이상 시청

In [ ]:
active = (target_pattern.시청일수 > 2) & (target_pattern.총시청시간 > 3600)
T2 = target_pattern[(target_pattern.기간 == 'T2')].copy()
T3 = target_pattern[(target_pattern.기간 == 'T3')].copy()
T4 = target_pattern[(target_pattern.기간 == 'T4')].copy()

각 기간 별 active audience list

In [ ]:
ids = overwatch.groupby('기간').ID.apply(set).reset_index().set_index('기간')

시간의 흐름에 따른 개인 시청자의 시청패턴 변화를 보기 위해서 T2, T3, T4 모두 active audience로 활동한 시청자를 target audience로 지정

In [ ]:
target_audience = list(reduce(set.intersection, ids.ID))
# target_audience = list(ids.ID[0] & ids.ID[1] & ids.ID[2])

### T2에서 SBS를 주로 시청하던 사람들은 무엇을 시청하던 사람들인가?

In [ ]:
t2_sbs = T2[T2[target_channel].idxmax(axis=1) == 'SBS'].ID.tolist()
t2_kbs1 = T2[T2[target_channel].idxmax(axis=1) == 'KBS1'].ID.tolist()
t2_mbc = T2[T2[target_channel].idxmax(axis=1) == 'MBC'].ID.tolist()
t2_jtbc = T2[T2[target_channel].idxmax(axis=1) == 'JTBC'].ID.tolist()
t2_kbs2 = T2[T2[target_channel].idxmax(axis=1) == 'KBS2'].ID.tolist()
t2_mbn = T2[T2[target_channel].idxmax(axis=1) == 'MBN'].ID.tolist()
# t2_tvJ = T2[T2[target_channel].idxmax(axis=1) == 'TV조선'].ID.tolist()

In [ ]:
genre = pattern.columns[15:23]

In [ ]:
# demo_df.loc[target_audience]
# demo_df.loc[t2_sbs].연령.plot(kind = 'hist' ,bins=np.arange(0,80,10))
plt.figure(figsize=(15,12))
sns.distplot(demo_info.loc[t2_sbs].연령, bins=np.arange(0,90,10), label='SBS', kde=True)
sns.distplot(demo_info.loc[t2_kbs1].연령, bins=np.arange(0,90,10), label='KBS1', kde=True)
sns.distplot(demo_info.loc[t2_kbs2].연령, bins=np.arange(0,90,10), label='KBS2', kde=True)
sns.distplot(demo_info.loc[t2_jtbc].연령, bins=np.arange(0,90,10), label='JTBC', kde=True)
sns.distplot(demo_info.loc[t2_mbc].연령, bins=np.arange(0,90,10), label='MBC', kde=True)
sns.distplot(demo_info.loc[t2_mbn].연령, bins=np.arange(0,90,10), label='MBN', kde=True,)
plt.legend();

## 3.3. 장르편향 및 선호장르
한 장르를 50% 이상 시청하면 장르 편향이 있다고 판단
T2, T3, T4에서 이 편향이 바뀌지 않으면 그 사람의 특성으로 파악

In [ ]:
picky_all = target_pattern[genre].max(axis=1) > 30
target_pattern['장르편향'] = picky_all

In [ ]:
target_pattern[genre]

In [ ]:
target_pattern['선호장르'] = target_pattern[genre].idxmax(axis=1)

### 3.3.1. 기간별로 각 개인의 선호장르가 변했는가 : genre_bias

In [ ]:
bias = target_pattern[['기간', 'ID', '장르편향', '선호장르']].sort_values(['ID', '기간'])

In [ ]:
genre_bias = pd.pivot_table(bias, columns = ['기간'], 
                            index=['ID'], values=['선호장르'], 
                            aggfunc=lambda x: ' '.join(x))

genre_bias.columns = genre_bias.columns.get_level_values(level=1)

In [ ]:
genre_bias['불변1'] = genre_bias.T2 == genre_bias.T3
genre_bias['불변2'] = genre_bias.T3 == genre_bias.T4
genre_bias['불변'] = genre_bias.불변1 & genre_bias.불변2

In [ ]:
genre_bias.불변.value_counts()

## 3.4. 프라임 시간대 뉴스 비교

In [ ]:
news = overwatch.프로그램명.apply(lambda x: '뉴스' in x)
dinner = (overwatch.프로그램편성시작시간.dt.hour >= 19) & (overwatch.프로그램편성시작시간.dt.hour <= 21)
min_duration = overwatch.프로그램시청시간 > 180

In [ ]:
agg5 = {
    '프로그램시청시간' : {
        '총시청시간합' : 'sum',
        '평균시청시간' : 'mean',
        '시청횟수' : 'count'
    },
    'ID' : {
        '시청자수' : lambda x: len(set(x))
    }
}

In [ ]:
dinner_news = overwatch[news & dinner & min_duration]\
.groupby(['프로그램명']).agg(agg5)

In [ ]:
dinner_news.columns = dinner_news.columns.get_level_values(level=1)

### 3.4.1. 전체기간 저녁시간대 뉴스 순위

#### 3.4.1.1. 시청자수 기준

In [ ]:
dinner_news.sort_values(by='시청자수', ascending=False).head(10)

#### 3.4.1.2. 평균 시청시간 기준

In [ ]:
dinner_news.sort_values(by='평균시청시간', ascending=False).head(10)

상위 저녁시간대 뉴스 순위 기준으로 main_news 선정

In [ ]:
main_news = dinner_news.sort_values(by='시청자수', ascending=False).head(5).index.tolist()

In [ ]:
main_news

In [ ]:
news_watch = overwatch[overwatch.프로그램명.isin(main_news) & min_duration].groupby(['기간', 'ID', '프로그램명']).agg(agg5)
news_watch.columns = news_watch.columns.get_level_values(level=1)

In [ ]:
news_watch.reset_index(inplace=True)

In [ ]:
main_news_watch= pd.pivot_table(news_watch, index=['기간','ID'], columns='프로그램명', values='총시청시간합').replace(np.nan, 0)

In [ ]:
main_news_watch

In [ ]:
idx = target_pattern.set_index(['기간','ID']).index

In [ ]:
main_news_df = pd.DataFrame(index = idx)

In [ ]:
main_news_df = pd.merge(left= main_news_df.reset_index(), right=main_news_watch.reset_index(), how='left')\
    .sort_values(by=['ID', '기간']).set_index(['기간', 'ID'])

In [ ]:
main_news_df['주시청뉴스'] = main_news_df.idxmax(axis=1).replace(np.nan, '미시청')

In [ ]:
main_news_final = pd.pivot_table(main_news_df.reset_index(), index='ID', columns='기간', values='주시청뉴스', aggfunc=lambda x: ''.join(x))

In [ ]:
main_news_final.loc[GONE]

In [ ]:
T2_SBS_NEWS = main_news_final[main_news_final.T2 == 'SBS8뉴스'].index.tolist()
T3_SBS_NEWS = main_news_final[main_news_final.T3 == 'SBS8뉴스'].index.tolist()
T4_SBS_NEWS = main_news_final[main_news_final.T4 == 'SBS8뉴스'].index.tolist()

In [ ]:
len(T4_SBS_NEWS)

In [ ]:
GONE = main_news_final[(main_news_final.T2 == 'SBS8뉴스') & 
                ((main_news_final.T3 != 'SBS8뉴스') &
                (main_news_final.T4 != 'SBS8뉴스') |
                (main_news_final.T3 == 'SBS8뉴스') &
                (main_news_final.T4 != 'SBS8뉴스'))].index.tolist()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(demo_info.loc[T2_SBS_NEWS].연령, bins=np.arange(0,90,10), label='SBS8뉴스', kde=False)
sns.distplot(demo_info.loc[GONE].연령, bins=np.arange(0,90,10), label='이탈', kde=False)
plt.legend()

### Sankey Diagram을 위한 데이터 전처리

In [ ]:
main_news_final.T2 = 'T2_' + main_news_final.T2
main_news_final.T3 = 'T3_' + main_news_final.T3
main_news_final.T4 = 'T4_' + main_news_final.T4

In [ ]:
main_news_final.reset_index().groupby(['T2', 'T3']).agg({'ID': lambda x : len(x)})\
    .reset_index().to_csv('output/news_t2_t3.csv', index=False, encoding='utf8')

In [ ]:
main_news_final.reset_index().groupby(['T3', 'T4']).agg({'ID': lambda x : len(x)})\
    .reset_index().to_csv('output/news_t3_t4.csv', index=False,encoding='utf8')